In [1]:
import pandas as pd
import numpy as np
from automation_process_dataV2 import united_functions
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

In [2]:
df_rating = pd.read_parquet('../../data/raw/data_model/ratings_train05.parquet')#Carga de datos

In [3]:
# muestrear 0.5% de regitros
_, subset_rating = train_test_split(df_rating, test_size=0.005, random_state=111)

In [4]:
# Guardar muestra para aplicarla en la app final
subset_rating.to_parquet('../../data/raw/data_model/final_data_used/subset_splitV2.parquet')

In [5]:
subset_rating.reset_index(inplace=True, drop=True)

In [6]:
subset_rating.shape

(5001, 6)

In [7]:
df_processed, list_user_movies= united_functions(subset_rating)# Script de procesamiento de datos y retorno de titulo con id

INICIANDO PROCESAMIENTO DE DATOS...
- Eliminacion de nulos..
- Eliminando variables no necesarias...
- Extrayendo año del titulo...
- Almacenando en variable copia del df original...
- Extraemos los generos de cada pelicula...
- Eliminamos variables no necesarias...
- Codificamos los generos y normalizamos para su correcto formato...
FIN DE PROCESAMIENTO DE DATOS


In [8]:
df_processed.shape

(5001, 21)

In [11]:
# Guardar resutado de procesamiento para aplicarla en la app final
df_processed.to_csv('../../data/raw/data_model/final_data_used/process_splitV2.csv', index=False)
list_user_movies.to_csv('../../data/raw/data_model/final_data_used/list_user_moviesV2.csv', index=False)

In [12]:
# Diccionario de list_user_movies
dicc_user_movies = {'userId':list_user_movies['userId'], 'movieId':list_user_movies['movieId'], 'title':list_user_movies['title']}

Similitud del coseno (compara cada pelicula con todas las encontradas)

In [13]:
simil_cosine=cosine_similarity(df_processed)# calculo de similitus del coseno

In [14]:
# Rellenamos la diagonal con NaN y convertimos a dataframe
np.fill_diagonal(simil_cosine, np.nan)
df_similCosine = pd.DataFrame(simil_cosine)

In [15]:
df_similCosine

,0,1,2,3,4,5,6,7,8,9,...,4991,4992,4993,4994,4995,4996,4997,4998,4999,5000
0,NaN,0.843520,0.600872,0.288845,0.538716,0.483019,0.702642,0.590991,0.847506,0.497009,...,0.635295,0.563602,0.614177,0.364697,0.583594,0.542506,0.543597,0.702642,0.337237,0.475542
1,0.843520,NaN,0.726251,0.309750,0.638247,0.291717,0.821787,0.349481,0.990687,0.588802,...,0.340158,0.661471,0.378941,0.409325,0.684010,0.329371,0.289596,0.821787,0.354283,0.511865
2,0.600872,0.726251,NaN,0.166347,0.502494,0.150260,0.603828,0.187374,0.721732,0.424651,...,0.216809,0.513020,0.211906,0.191095,0.519683,0.180307,0.148670,0.603828,0.201863,0.271890
3,0.288845,0.309750,0.166347,NaN,0.167459,0.543172,0.264610,0.665741,0.324641,0.549462,...,0.212583,0.418075,0.450183,0.592248,0.204681,0.399952,0.649301,0.264610,0.440071,0.624581
4,0.538716,0.638247,0.502494,0.167459,NaN,0.146580,0.542783,0.188401,0.648305,0.585161,...,0.243236,0.730695,0.219470,0.171351,0.468416,0.184018,0.144650,0.542783,0.211701,0.271513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4996,0.542506,0.329371,0.180307,0.399952,0.184018,0.727130,0.285647,0.672131,0.349585,0.554744,...,0.739922,0.207153,0.680092,0.248902,0.445760,NaN,0.650533,0.285647,0.454871,0.329371
4997,0.543597,0.289596,0.148670,0.649301,0.144650,0.812346,0.238834,0.993233,0.294749,0.386872,...,0.471421,0.162515,0.720821,0.245534,0.179968,0.650533,NaN,0.238834,0.193194,0.289596
4998,0.702642,0.821787,0.603828,0.264610,0.542783,0.241209,0.691175,0.298165,0.830999,0.482972,...,0.333215,0.566051,0.334210,0.313796,0.581445,0.285647,0.238834,NaN,0.317141,0.433524
4999,0.337237,0.354283,0.201863,0.440071,0.211701,0.195313,0.592314,0.244335,0.386158,0.610995,...,0.286219,0.490946,0.277212,0.630310,0.750582,0.454871,0.193194,0.317141,NaN,0.700238


In [16]:
# Guardar matriz similitudes coseno
df_similCosine.to_csv('cosine_csv/df_similCosineV2.csv', index=False)

Logica que implementa la recomendacion en base a un usuario y pelicula vista

In [16]:
def comprobar_existencia(diccionario, movie_id, user_id):    
    for i in range(len(diccionario['userId'])):
        if diccionario['userId'].iloc[i] == user_id and diccionario['movieId'].iloc[i] == movie_id:
            return True
    return False

In [17]:
def obtener_index_movieId(diccionario, movie_id):
    for i in range(len(diccionario['movieId'])):
        if diccionario['movieId'].iloc[i]==movie_id:
            index = i
            break
    return index

In [18]:
def get_title_from_movieId(user_id ,diccionario):
    index = diccionario['userId'].loc[diccionario['userId'] == user_id].index
    # Obtén los títulos de las películas
    titles_saws = diccionario['title'].loc[index]
    return titles_saws

In [19]:
# Funcion de recomendacion para un usuario y pelicula
def get_recommendations(user_id, movie_id,dicc_user_movies=dicc_user_movies, simil_cosine=df_similCosine, threshold=0.7):
    #Comprobamos que el usuario para la pelicula dada existe
    comprobar_user_movie_exist=comprobar_existencia(dicc_user_movies, movie_id, user_id)

    if comprobar_user_movie_exist:
       #Obtener indice de la pelicula dada
        index=obtener_index_movieId(dicc_user_movies, movie_id)

        # Obtén la lista de similitudes del coseno para una película y usuario dada
        cosine_sim_list = simil_cosine.iloc[index]
        
        # Ordena las películas en función de la similitud del coseno
        most_similar_movies_sorted=cosine_sim_list.sort_values(ascending=False)

        # Obtiene los índices de las 5 películas más similares superior al umbral 
        most_similar_movies = most_similar_movies_sorted[most_similar_movies_sorted > threshold][:5].index

        # Obtiene los títulos de las películas más similares
        most_similar_movie_titles = list(set([dicc_user_movies['title'][id] for id in most_similar_movies if id in dicc_user_movies['title']]))
        
        # Obtener los titulos de todas las peliculas vistas por el usuario
        titles_saws = get_title_from_movieId(user_id ,dicc_user_movies)
        titles_saws = list(titles_saws)
        
        # Pelicula actual pasada por parametros (title)
        actual_title = dicc_user_movies['movieId'].loc[dicc_user_movies['movieId'] == movie_id].index[0]

        print('Porque has visto:',dicc_user_movies['title'].loc[actual_title])
        
        # Filtra las películas recomendadas para excluir las que el usuario ya ha visto
        recommended_movies = [movie for movie in most_similar_movie_titles if movie not in titles_saws]

        # Si las peliculas se devuelven vacias se retorna la condicion
        if not recommended_movies:
            return 'No se encontro una pelicula lo suficientemente significativa para recomendar'
        print('Recomendamos:')
        return recommended_movies
    else: 
        return 'No se encontro usuario para pelicula dada'
        

In [21]:
result_recommendation=get_recommendations(47985,33004)#userId, movieId
result_recommendation

Porque has visto: Hitchhiker's Guide to the Galaxy, The
Recomendamos:


['Galaxy Quest',
 "Bill & Ted's Excellent Adventure",
 'Back to the Future Part II']

Podemos ver que para la pelicula Hitchhiker's Guide to the Galaxy, The (cuyo genero es comedia/ciencia ficcion lanzada en el 2005), 

se recomiendan tres peliculas de comedia/ciencia, ambas lanzadas en años similares.

In [20]:
list_user_movies.tail(10)

,userId,movieId,rating,title
1991,47985.0,33004,4.5,"Hitchhiker's Guide to the Galaxy, The"
1992,116941.0,5401,4.0,Undercover Brother
1993,7405.0,2826,2.0,"13th Warrior, The"
1994,87208.0,2762,5.0,"Sixth Sense, The"
1995,53360.0,1617,5.0,L.A. Confidential
1996,83373.0,2206,3.5,Suspicion
1997,104655.0,2193,3.0,Willow
1998,50541.0,3081,3.5,Sleepy Hollow
1999,11984.0,700,3.0,Angus
2000,27783.0,1245,5.0,Miller's Crossing


In [17]:
# Buscar usuarios 
def movies_peruser(idUser):    
    print(list_user_movies[list_user_movies['userId']==idUser])

In [24]:
movies_peruser(136540)

         userId  movieId  rating             title
19995  136540.0       36     3.0  Dead Man Walking
